## Check currently running jobs

The `sfapi_client` can easily be used to get your current jobs running on the system, or information about past jobs.

First we'll import the required libraries, `Client`  to handle the requests and the `Machines` enum to get the correct resource.

In [13]:
# import client library
from sfapi_client import Client
# import the machines enum to choose the resource
from sfapi_client.compute import Machines
# this will help display the outputs later
import json

user_name="elvis"

The next lines of code will create a client which can be used to get an `Compute` object. This object can be used to look at the current and past job queues, list files and directories, and run simple commands in the system.

In this example we will get the currently running jobs for the user `elvis`.

In [2]:
with Client() as client:
    perlmutter = client.compute(Machines.perlmutter)
    # This selects just the jobs in the regular cpu partition on perlmutter
    jobs = perlmutter.jobs(user=user_name, partition='regular_milan_ss11')

All the jobs currenlty in the job queue are stored in the newly created `jobs` list. An easy way of viewing and this list is to print the objects in the list. 

In [3]:
print(json.dumps([j.dict() for j in jobs], indent=4))

[
    {
        "account": "ntrain",
        "tres_per_node": "N/A",
        "min_cpus": "2",
        "min_tmp_disk": "0",
        "end_time": "2023-04-27T15:52:57",
        "features": "cpu",
        "group": "12345",
        "over_subscribe": "NO",
        "jobid": "8218256",
        "name": "HPXGaudiTest",
        "comment": "(null)",
        "time_limit": "30:00",
        "min_memory": "0",
        "req_nodes": "",
        "command": "/global/cfs/cdirs/ntrain/servicenow/INC0200856/perlmutter-podman.sh",
        "priority": "67679",
        "qos": "regular_1",
        "reason": "None",
        "field_": null,
        "st": "R",
        "user": "elvis",
        "reservation": "(null)",
        "wckey": "(null)",
        "exc_nodes": "",
        "nice": "0",
        "s_c_t": "*:*:*",
        "exec_host": "nid005656",
        "cpus": "2048",
        "nodes": "8",
        "dependency": "(null)",
        "array_job_id": "8218256",
        "sockets_per_node": "*",
        "cores_per_socke

A single job can be pulled from the list for further inspection. This returns a `Squeue` object which is the output you would get from running the command `squeue` from slurm. This is useful for getting jobs which are currently pending or running.

In [4]:
jobs[0]

JobSqueue(account='ntrain', tres_per_node='N/A', min_cpus='2', min_tmp_disk='0', end_time='2023-04-27T15:52:57', features='cpu', group='12345', over_subscribe='NO', jobid='8218256', name='HPXGaudiTest', comment='(null)', time_limit='30:00', min_memory='0', req_nodes='', command='/global/cfs/cdirs/ntrain/servicenow/INC0200856/perlmutter-podman.sh', priority='67679', qos='regular_1', reason='None', field_=None, st='R', user='elvis', reservation='(null)', wckey='(null)', exc_nodes='', nice='0', s_c_t='*:*:*', exec_host='nid005656', cpus='2048', nodes='8', dependency='(null)', array_job_id='8218256', sockets_per_node='*', cores_per_socket='*', threads_per_core='*', array_task_id='N/A', time_left='3:13', time='26:47', nodelist='nid[005656-005663]', contiguous='0', partition='regular_milan_ss11', nodelist_reason_='nid[005656-005663]', start_time='2023-04-27T15:22:57', state=<JobState.RUNNING: 'RUNNING'>, uid='12345', submit_time='2023-04-27T15:20:28', licenses='cfs:1', core_spec='N/A', sched

More specific information about the job can also be gotten from this object like the number of nodes or it's `jobid`.

In [9]:
print(f"Numer of nodes = {jobs[0].nodes}")
print(f"jobid = {jobs[0].jobid}")

Numer of nodes = 8
jobid = 8218256


For jobs that have finished it is useful to get infomation about them using the `sacct` command which can be done in the sfapi_client library requesting just one `jobid` or by specifying the command in the job request.

In [11]:
with Client() as client:
    perlmutter = client.compute(Machines.perlmutter)
    # This selects just the jobs in the regular cpu partition on perlmutter
    job = perlmutter.job(jobid=jobs[0].jobid)

In [12]:
job

JobSacct(account='ntrain', admincomment='{"stdinPath":"\\/dev\\/null","arrayTaskId":4294967294,"qos":"regular_1","nodes":"nid[005656-005663]","jobDerivedExitCode":0,"allocNodes":8,"licenses":"cfs:1","jobExitCode":0,"cluster":"perlmutter","features":"cpu","batchHost":"nid005656","timeLimit":30,"submitTime":1682634028,"packJobId":0,"argv":["\\/global\\/cfs\\/cdirs\\/ntrain\\/servicenow\\/INC0200856\\/perlmutter-podman.sh"],"tresRequest":"1=2048,2=3904016,3=18446744073709551614,4=8,5=2048","gresRequest":"cpu=16,mem=3904016M,node=8,billing=16","arrayJobId":0,"name":"HPXGaudiTest","packJobOffset":0,"partition":"regular_milan_ss11","workingDirectory":"\\/global\\/cfs\\/cdirs\\/ntrain\\/servicenow\\/INC0200856","jobAccount":"ntrain","restartCnt":0,"reboot":0,"uid":12345,"endTime":1682636006,"allocCpus":2048,"jobId":8218256,"startTime":1682634177,"priority":67679,"resizing":0}', alloccpus='2048', allocnodes='8', alloctres='billing=2048,cpu=2048,energy=3774233,mem=3904016M,node=8', associd='206